In [1]:
from configparser import ConfigParser
import cv2
import math
import numpy as np
import os
import pandas as pd
from pprint import pprint

In [2]:
## constants
############

CONFIG_PATH = './conf/config.ini'

In [3]:
## config
#########

config = ConfigParser()

config.read(CONFIG_PATH)
sequence_config = dict(config['Sequence'])
tracking_config = dict(config['Tracking'])

sequence_config['frame_rate'] = config.getint("Sequence","frame_rate")
sequence_config['seq_length'] = config.getint("Sequence","seq_length")
sequence_config['img_width'] = config.getint("Sequence","img_width")
sequence_config['img_height'] = config.getint("Sequence","img_height")

tracking_config['scaled_height'] = config.getint("Tracking","scaled_height")
tracking_config['scaled_width'] = config.getint("Tracking","scaled_width")
tracking_config['memory'] = config.getint("Tracking","memory")
tracking_config['similarity_threshold'] = config.getfloat("Tracking","similarity_threshold")
tracking_config['confidence_threshold'] = config.getfloat("Tracking","confidence_threshold")

In [4]:
## input
########

# pos       name                    description
# 1      frame number           frame in which the object is present
# 2      identity number        trajectory id (-1 default for no track)
# 3      bounding box x         x value from top left of bounding box
# 4      bounding box y         y value from top left of bounding box
# 5      bounding box width     width of bounding box in pixels
# 6      bounding box height    height of bounding box in pixels
# 7      confidence score      class detection confidence

In [5]:
## output
#########

# pos       name                    description
# 1      frame number           frame in which the object is present
# 2      identity number        trajectory id (-1 default for no track)
# 3      bounding box x         x value from top left of bounding box
# 4      bounding box y         y value from top left of bounding box
# 5      bounding box width     width of bounding box in pixels
# 6      bounding box height    height of bounding box in pixels
# 7      confidence score*      class detection confidence (gt: 1 or 0)
# 8      class*                 type of class (1 for pedestrian)
# 9      visibility*            percent visible (percent occluded = 1-visibility)

#        *no need to output these values, will be ignore by evaluator

In [6]:
class Point(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [7]:
class Distance(object):
    
    @staticmethod
    def l2_norm(obj1, obj2):
        p1, p2 = None, None
        
        if type(obj1) == Box:
            p1 = obj1.centroid
        elif type(obj1) == Point:
            p1 = obj1
        
        if type(obj1) == Box:
            p2 = obj2.centroid
        elif type(obj1) == Point:
            p2 = obj2
        
        return math.sqrt((p2.x-p1.x)**2 + (p2.y-p1.y)**2)
    
    @staticmethod
    def jaccard(obj1, obj2):
        
        if type(obj1) == Box and type(obj2) == Box:
            box1, box2 = obj1, obj2
            
            w_intersection = min(box1.x + box1.w, box2.x + box2.w) - max(box1.x, box2.x)
            h_intersection = min(box1.y + box1.h, box2.y + box2.y) - max(box1.y, box2.y)
            if w_intersection <= 0 or h_intersection <= 0: # No overlap
                return 0
            I = w_intersection * h_intersection
            U = box1.w * box1.h + box2.w * box2.h - I # Union = Total Area - I
            return I / U
    
        elif type(obj1) == numpy.ndarray and type(obj2) == numpy.ndarray:
            descriptors1, descriptors2 = obj1, obj2
            
            # brute force feature matching using manhattan distance
        
            bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

            matches = bf.match(descriptors_1,descriptors_2)
            matches = sorted(matches, key = lambda x:x.distance)

            # TODO do we want to cutoff at a threshold?

            return len(matches) / (len(descriptors1)+len(descriptors2))
    
    @staticmethod
    def sift(box1, box2, img_path, frame_a_id, frame_b_id, new_w=tracking_config['scaled_width'], new_h=tracking_config['scaled_height']):
        
        # get filepaths
        
        img1_path = os.path.join(img_path, "%06d.jpg" % frame_a_id)
        img2_path = os.path.join(img_path, "%06d.jpg" % frame_b_id)
        
        # read image files as grayscale
        
        img1 = cv2.imread(img1_path, 0)
        img2 = cv2.imread(img2_path, 0)
        
#         # gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
#         # gray2 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        
        # cut out the firt object from the first image
        
        top_left, bottom_right = box1.coords        
        start_x, start_y = top_left
        end_x, end_y = bottom_right        
        crop1 = img1[start_y:end_y, start_x:end_x]
        
        # cut out the second object from the second image
        
        top_left, bottom_right = box2.coords        
        start_x, start_y = top_left
        end_x, end_y = bottom_right        
        crop2 = img2[start_y:end_y, start_x:end_x]
        
#         # calculate the amount to scale in both directions
        
#         img_1_x_scale = new_w/box1.w
#         img_1_y_scale = new_h/box1.h
        
#         img_1_x_scale = new_w/box2.w
#         img_1_y_scale = new_h/box2.h
        
#         # resize both cropped images
        
#         scaled_img1 = cv2.resize(img1, None, fx=img_1_x_scale, fy=img_1_y_scale, interpolation = cv2.INTER_CUBIC)
#         scaled_img2 = cv2.resize(img2, None, fx=img_2_x_scale, fy=img_2_y_scale, interpolation = cv2.INTER_CUBIC)

#         # TODO do I even need to scale? SIFT is Scale Invariant

#         scaled_img1 = cv2.resize(img1, (new_w, new_h), interpolation = cv2.INTER_CUBIC)
#         scaled_img2 = cv2.resize(img2, (new_w, new_h), interpolation = cv2.INTER_CUBIC)
        
#         cv2.imshow("img1", scaled_img1)
#         cv2.imshow("img2", scaled_img2)
        
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        
        # find keypoints and descriptors
        
        # https://docs.opencv.org/4.3.0/da/df5/tutorial_py_sift_intro.html
        
        sift = cv2.xfeatures2d.SIFT_create()
        
        keypoints_1, descriptors_1 = sift.detectAndCompute(scaled_img1,None)
        keypoints_2, descriptors_2 = sift.detectAndCompute(scaled_img2,None)

        # brute force feature matching using manhattan distance
        
        bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

        matches = bf.match(descriptors_1,descriptors_2)
        matches = sorted(matches, key = lambda x:x.distance)
        
        # TODO do we want to cutoff at a threshold?
        
        pprint(matches)
        pprint(keypoints1)
        pprint(keypoints2)
        
        return len(matches) / (len(descriptors_1)+len(descriptors_2))
        pass


In [8]:
class Box(object):
    def __init__(self, x, y, w, h, index=None, frame=None, track_id=None, conf=None):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.frame = frame
        self.index = index
        self.track_id = track_id
        self.conf = conf
    
    @property
    def coords(self):
        return(Point(self.x, self.y), Point(self.x+self.w, self.y+self.h))
    
    @property
    def centroid(self):
        return Point(self.x+(self.w*0.5), self.y+(self.h*0.5))
    
    # TODO: move the similarity/distance calculation out of Box, doesn't belong here
    
    def jaccard(self, box):
        w_intersection = min(self.x + self.w, box.x + box.w) - max(self.x, box.x)
        h_intersection = min(self.y + self.h, box.y + box.y) - max(self.y, box.y)
        if w_intersection <= 0 or h_intersection <= 0: # No overlap
            return 0
        I = w_intersection * h_intersection
        U = self.w * self.h + box.w * box.h - I # Union = Total Area - I
        return I / U

    def copy(self, offset_x=0, offset_y=0):
        return Box(
            self.x + offset_x, 
            self.y + offset_y, 
            self.w, self.h, 
            self.index, self.frame,
            self.track_id, self.conf
        )
    
    def __repr__(self):
        return "Box(%s, %s, %s, %s, %s, %s, %s, %s)" % (
            self.x, self.y, self.w, self.h, self.index, self.frame, self.track_id, self.conf
        )
    
    def __str__(self):
        return "index: %s, frame: %s, track: %s, x: %s, y: %s, w: %s, h: %s, conf: %s" % (
            self.index, self.frame, self.track_id, self.x, self.y, self.w, self.h, self.conf
        )

In [9]:
class Track(object):
    counter = 1
    
    def __init__(self, o):
        self.id = Track.counter
        Track.counter += 1
        
        if type(o) == list:
            boxes = o
            self.boxes = boxes
        elif type(o) == Box:
            box = o
            self.boxes = [box]
            
        self.is_active = True
    
    def add(self, box):
        box.track_id = self.id
        self.boxes.append(box)
    
    @staticmethod
    def angle(box1, box2):
        
        p1 = box1.centroid
        p2 = box2.centroid

        rads = math.atan2(p1.y-p2.y, p1.x-p2.x)
        # deg = math.degrees(rads)
        # return rads
        return rads
    
    @staticmethod
    def distance(box1, box2):
            
        p1 = box1.centroid
        p2 = box2.centroid

        distance = math.hypot((p2.x-p1.x),(p2.y-p1.y))
        # distance = sqrt((x2-x1)**2 + (y2-y1)**2)
        
        return distance
    
    def predict(self, frame_id):
        current = self.boxes[-1]
        
        if (frame_id - current.frame) > tracking_config['memory']:
        # if current.frame != frame_id - 1:
            return None
        
        if len(self.boxes)==1:
            predicted_location = current.copy()
        else:
            previous = self.boxes[-2]
            
            angle = Track.angle(previous, current)
            distance = Track.distance(previous, current)
            
            offset_x = distance*math.sin(angle)
            offset_y = distance*math.cos(angle)
            
            # predicted_location = current.copy() # match last location
            predicted_location = current.copy(offset_x, offset_y) # estimate trajectory
            
        predicted_location.frame += 1

        return predicted_location

    def __repr__(self):
        return "Track([<__main__.Box>]) count: %s" % len(self.boxes)
    
    def __str__(self):
        return "TrackID: %s, Frames: %s-%s" % (
            self.id, self.boxes[0].frame, self.boxes[-1].frame
        )
    
    def __len__(self):
        return len(self._boxes)

In [10]:
class Frame(object):
    def __init__(self, number, boxes=None):
        self.id = number
        
        if boxes is None:
            boxes = []
        
        self._boxes = boxes
    
    def add(self, box):
        if type(box) == Box:
            self._boxes.append(box)
        elif type(box) == list:
            self._boxes.extend(boxes)
    
    @property
    def objects(self):
        return self._boxes
    
    def __repr__(self):
        return "Frame(%s, [<__main__.Box>]) count: %s" % (self.id, len(self.boxes))
    
    def __str__(self):
        return "FrameID: %s, Count: %s" % (self.id, len(self.boxes))
    
    def __len__(self):
        return len(self._boxes)

In [11]:
class Detections(object):
    def __init__(self, data_path=tracking_config['detections_path']):
        self.data_path = data_path
        self.mean_width = None
        self.mean_height = None
        self._df = None
        self._frames = {}
        self.start_index = None
        self.end_index = None
        self.count = sequence_config['seq_length']
        self._pos = None
        self._load()
    
    def _load(self, conf_threshold=tracking_config['confidence_threshold']):
        
        header_list = ['frame','trajectory','x','y','w','h','confidence']
        dtype = {'frame':int,'trajectory':int,'x':float,'y':float,'w':float,'h':float,'confidence':float}
        
        df = pd.read_csv(self.data_path, names=header_list, dtype=dtype)

        # filter out low confidence detections
        
        self._df = df[df["confidence"] >= conf_threshold]
        
        self.mean_width = self._df["w"].mean()
        self.mean_height = self._df["h"].mean()

        # get the indices for the first and last frames

        self.start_index = self._df['frame'].min()
        self.end_index = self._df['frame'].max() + 1

        for i in range(self.start_index, self.end_index):
            self._frames[i] = self._df.loc[df['frame']==i,:]
    
    def __iter__(self):
        return self
    
    def __next__(self):
        return self.next()
    
    def next(self):
        while True:
            if self._pos is None:
                self._pos = self.start_index        
            elif self._pos < self.end_index:
                current, self._pos = self._pos, self._pos + 1
                current_frame = self._frames[current]

                detection_boxes = []

                for i, detection in current_frame.iterrows():                
                    frame_no, traj_no, x, y, w, h, conf = detection
                    box = Box(x, y, w, h, i, frame_no, -1, conf)
                    detection_boxes.append(box)

                return Frame(current, detection_boxes)
            else:
                # self._pos = None
                raise StopIteration()
        
    def __repr__(self):
        return "Detections(%s, %s, %s, %s, %s, %s)" % (
            self.data_path, self.mean_width, self.mean_height, 
            self.start_index, self.end_index, self.count
        )
    
    def __str__(self):
        return "path: %s, name: %s, mean_width: %s, mean_height: %s, start_index: %s, end_index: %s, count: %s)" % (
            self.data_path, sequence_config['name'], 
            self.mean_width, self.mean_height, 
            self.start_index, self.end_index, self.count
        )
    
    def __len__(self):
        # TODO this is really the length of the frames, that's how
        # I use it in trajectories... should rethink this
        return self.count

In [12]:
class Trajectories(object):
    def __init__(self):
        # reset track counter
        Track.counter = 1
        self._tracks = {}
        self.detections = Detections()
    
    def add(self, o):
        if type(o) == Box:
            box = o
            track = Track(box)
            box.track_id = track.id
            self._tracks[track.id] = track
        if type(o) == Track:
            track = o
            self._tracks[track.id] = track
    
    def _matching(self, predictions, current_frame):
        measure = tracking_config['distance_measure']
        
        if measure == 'euclidean':
            distance_func = Distance.l2_norm
        elif measure == 'iou':
            distance_func = Distance.jaccard
        else:
            distance_func = Distance.sift
            
        sim_vector = []
        computed = []
        
        for box_a in predictions:
            for box_b in current_frame.objects:
                
                # WARNING this is kind of dangerous because index is not 
                # a required parameter for Box. Tracking computed pairs
                # is meant to save time, but it might save too much time
                # if box indices are None.
                if box_a.index is None or box_b.index is None:
                    raise Exception("Boundary index may not be None.")                    
                    
                this_pair = set([box_a.index, box_b.index])
                
                if this_pair not in computed:
                    # WARNING I think this is a problem because sift
                    # has unique parameters (img, frame, etc)
                    similarity = distance_func(box_a, box_b)
                    
                    if measure == 'euclidean':
                        if similarity <= tracking_config['similarity_threshold']:
                            sim_vector.append((similarity, box_a, box_b))
                    else:
                        if similarity >= tracking_config['similarity_threshold']:
                            sim_vector.append((similarity, box_a, box_b))
                        
                    computed.append(this_pair)

        if measure == 'euclidean':            
            sim_vector.sort(key=lambda x: x[0])
        else:
            sim_vector.sort(reverse=True)

        matching_pairs = []
        matched_objects = set([])

        for distance, box_1, box_2 in sim_vector:
            
            # we want to ensure that we only return the top matches, so we keep track
            # of the objects already matched and ignore less similar matches for these objects
            
            if box_1.index not in matched_objects and box_2.index not in matched_objects:                
            
                match = (box_1, box_2)
                matching_pairs.append(match)
                
                matched_objects.add(box_1.index)
                matched_objects.add(box_2.index)
        
        # unmatched detections in current frame need to become new tracks
        
#         for box_b in current_frame.objects:
#             if box_b.index not in matched_objects:
#                 self.add(box_b)

        # print(len(matched_objects), len(self._tracks))
        
        return matching_pairs
    
    def calculate(self, start=0, end=None):
        if end is None:
            end = len(self.detections)
            
        # for frame in self.detections[start:end]: # Detections not subscriptable
        
        for frame in self.detections:
            # TODO fix this mess with dataframes
            df = self.detections._frames[frame.id]
            
            if len(self._tracks)==0:
                for box in frame.objects:
                    self.add(box)
                    
                    # TODO fix this mess with dataframes
                    df.loc[box.index, "trajectory"] = box.track_id
                continue

            predictions = []
            for track_id, track in self._tracks.items():
                prediction = track.predict(frame.id)
                if prediction is not None:
                    predictions.append(prediction)

            matching_pairs = self._matching(predictions, frame)
            for box_1, box_2 in matching_pairs:
                track = self._tracks[box_1.track_id]
                track.add(box_2)
                
                # TODO fix this mess with dataframes
                df.loc[box_2.index, "trajectory"] = box_1.track_id

    def output(self, path=None):
        if path is None:            
            for track_id, boxes in self._tracks.items():
                print(track_id, boxes)
        else:
            # write
            pass
    
    def __len__(self):
        return len(self._tracks)

In [13]:
trajectories = Trajectories()
trajectories.calculate()
trajectories.output()

/home/bosworth/anaconda3/envs/motaustin-env/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


1 TrackID: 1, Frames: 1.0-125.0
2 TrackID: 2, Frames: 1.0-451.0
3 TrackID: 3, Frames: 1.0-1050.0
4 TrackID: 4, Frames: 1.0-390.0
5 TrackID: 5, Frames: 1.0-543.0
6 TrackID: 6, Frames: 1.0-1050.0
7 TrackID: 7, Frames: 1.0-560.0
8 TrackID: 8, Frames: 1.0-66.0
9 TrackID: 9, Frames: 1.0-38.0
10 TrackID: 10, Frames: 1.0-1050.0
11 TrackID: 11, Frames: 1.0-756.0
12 TrackID: 12, Frames: 1.0-385.0
13 TrackID: 13, Frames: 1.0-1050.0
14 TrackID: 14, Frames: 1.0-1050.0
15 TrackID: 15, Frames: 1.0-1050.0
16 TrackID: 16, Frames: 1.0-15.0
17 TrackID: 17, Frames: 1.0-1050.0
18 TrackID: 18, Frames: 1.0-520.0
19 TrackID: 19, Frames: 1.0-174.0
20 TrackID: 20, Frames: 1.0-635.0
21 TrackID: 21, Frames: 1.0-220.0
22 TrackID: 22, Frames: 1.0-1050.0
23 TrackID: 23, Frames: 1.0-38.0
24 TrackID: 24, Frames: 1.0-17.0
25 TrackID: 25, Frames: 1.0-628.0
26 TrackID: 26, Frames: 1.0-3.0


In [14]:
from random import randint
colors = []
offset = 50
for i in range(100):
    r = randint(0,255)
    g = randint(0,255)
    b = randint(0,255)
    
    if r < offset or g < offset or b < offset:
        colors.append(((r,g,b),(r+offset, g+offset, b+offset)))
    else:
        colors.append(((r,g,b),(r-offset, g-offset, b-offset)))

In [ ]:
video_frames = []
img_path = "img1"

img_filenames = [f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))]
img_filenames.sort()

for img_filename in img_filenames:
    frame_id = int(img_filename.split('.')[0])
    frame = trajectories.detections._frames[frame_id]
    img = cv2.imread(os.path.join(img_path, img_filename))
    for index, (frame,track,x,y,w,h,conf) in frame.iterrows():
        
        track, x, y, w, h = int(track), int(x), int(y), int(w), int(h)
        if track < 1:
            continue
        
        color_index = track % 100
        box_color, text_color = colors[color_index]
        
        top_left = (x, y)
        bottom_right = (x+w, y+h)
        thickness = 2
        
        cv2.rectangle(img, top_left, bottom_right, box_color, thickness)
        
        font = cv2.FONT_HERSHEY_SIMPLEX
        anchor_point = (x, y+20)
        scale = 0.75
        thickness = 2
        line_type = cv2.LINE_AA
        
        cv2.putText(img, str(track), anchor_point, font, scale, text_color, thickness, line_type)
        
    video_frames.append(img)

############ debug
first, second, third = video_frames[:3]
cv2.imwrite('output/first.jpg', first) 
cv2.imwrite('output/second.jpg', second) 
cv2.imwrite('output/third.jpg', third) 
##################

output_path = 'output/%s.mp4' % tracking_config['out_seq_name']
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
fps = 30.0
width = 1920
height = 1080
video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

for video_frame in video_frames:
    video_writer.write(video_frame)

video_writer.release()
video_frames.clear()